In [2]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import joblib
import pickle
import os
import wandb
import warnings
warnings.filterwarnings('ignore')

In [4]:
DATA_PATH = "/content/group_project/clean_data.csv"
SAVE_DIR = "/content/group_project/"
OBSERVATION_DAYS = 30
PREDICTION_DAYS = 31

In [5]:
try:
    from google.colab import userdata
    WANDB_API_KEY = userdata.get('WANDB')
    wandb.login(key=WANDB_API_KEY)
except:
    pass

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: katrinpochtar to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Data split

In [6]:
def create_temporal_dataset(df, observation_days=30, prediction_days=31):
    df = df.copy()
    df['event_dt'] = pd.to_datetime(df['event_dt'])

    max_date = df['event_dt'].max()
    min_date = df['event_dt'].min()

    observation_end = max_date - pd.Timedelta(days=prediction_days)
    observation_start = observation_end - pd.Timedelta(days=observation_days)

    if observation_start < min_date:
        observation_start = min_date

    print(f"\nObservation: {observation_start.date()} to {observation_end.date()}")
    print(f"Prediction: {observation_end.date()} to {max_date.date()}")

    observation_data = df[
        (df['event_dt'] >= observation_start) &
        (df['event_dt'] < observation_end)
    ].copy()

    prediction_data = df[df['event_dt'] >= observation_end]

    active_in_observation = set(observation_data['device_id'].unique())
    active_in_prediction = set(prediction_data['device_id'].unique())
    churned_users = active_in_observation - active_in_prediction

    print(f"Users: {len(active_in_observation):,} | Churned: {len(churned_users):,} ({len(churned_users)/len(active_in_observation):.1%})")

    churn_labels = {uid: 1 if uid in churned_users else 0 for uid in active_in_observation}

    return observation_data, churn_labels

def create_unified_split(observation_data, churn_labels, save_dir):
    user_ids = list(churn_labels.keys())
    labels = [churn_labels[uid] for uid in user_ids]

    train_val_ids, test_ids, train_val_labels, test_labels = train_test_split(
        user_ids, labels, test_size=0.2, random_state=42, stratify=labels
    )

    train_ids, val_ids, train_labels, val_labels = train_test_split(
        train_val_ids, train_val_labels, test_size=0.25, random_state=42, stratify=train_val_labels
    )

    split_data = {
        'train_ids': train_ids,
        'val_ids': val_ids,
        'test_ids': test_ids,
        'train_labels': train_labels,
        'val_labels': val_labels,
        'test_labels': test_labels
    }

    os.makedirs(save_dir, exist_ok=True)
    with open(os.path.join(save_dir, 'data_split.pkl'), 'wb') as f:
        pickle.dump(split_data, f)

    print(f"\nSplit: Train {len(train_ids):,} | Val {len(val_ids):,} | Test {len(test_ids):,}")

    return split_data

# Baseline

In [7]:
def extract_baseline_features(df, user_ids):
    event_features = df.groupby('device_id').agg({
        'event_dt': ['min', 'max', 'count'],
        'session_id': 'nunique',
        'screen': 'nunique',
        'feature': 'nunique',
        'action': 'count'
    })
    event_features.columns = ['_'.join(col).strip() for col in event_features.columns]

    event_features['days_in_window'] = (
        event_features['event_dt_max'] - event_features['event_dt_min']
    ).dt.total_seconds() / 86400

    event_features['events_per_day'] = event_features['event_dt_count'] / event_features['days_in_window'].clip(lower=1)
    event_features['sessions_per_day'] = event_features['session_id_nunique'] / event_features['days_in_window'].clip(lower=1)
    event_features['events_per_session'] = event_features['event_dt_count'] / event_features['session_id_nunique'].clip(lower=1)
    event_features['screen_diversity'] = event_features['screen_nunique'] / event_features['event_dt_count']
    event_features['feature_diversity'] = event_features['feature_nunique'] / event_features['event_dt_count']

    first_day = event_features['event_dt_min'].min()
    last_day = event_features['event_dt_max'].max()
    event_features['days_since_first_seen'] = (event_features['event_dt_min'] - first_day).dt.total_seconds() / 86400
    event_features['days_until_window_end'] = (last_day - event_features['event_dt_max']).dt.total_seconds() / 86400
    event_features['recency_in_window'] = event_features['days_until_window_end'] / event_features['days_in_window'].clip(lower=1)

    demographic = df.groupby('device_id')[['age', 'gender']].first()
    features = event_features.join(demographic, how='left')
    features = features.drop(columns=['event_dt_min', 'event_dt_max'])

    if 'gender' in features.columns:
        features = pd.get_dummies(features, columns=['gender'], prefix='gender', drop_first=False)

    features = features.fillna(features.median())
    features = features.loc[features.index.intersection(user_ids)]

    return features

def train_baseline(X_train, X_val, X_test, y_train, y_val, y_test, save_dir, config=None):
    run = wandb.init(project="churn-prediction", config=config)
    config = wandb.config

    model_type = getattr(config, 'model_type', 'xgboost')
    n_estimators = getattr(config, 'n_estimators', 200)
    max_depth = getattr(config, 'max_depth', 6)
    learning_rate = getattr(config, 'learning_rate', 0.1)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    if model_type == 'xgboost':
        model = XGBClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
            random_state=42
        )
    else:
        model = LGBMClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
            random_state=42,
            verbose=-1
        )

    model.fit(X_train_scaled, y_train)

    y_train_prob = model.predict_proba(X_train_scaled)[:, 1]
    y_val_prob = model.predict_proba(X_val_scaled)[:, 1]
    y_test_prob = model.predict_proba(X_test_scaled)[:, 1]

    y_val_pred = (y_val_prob > 0.5).astype(int)
    y_test_pred = (y_test_prob > 0.5).astype(int)

    train_auc = roc_auc_score(y_train, y_train_prob)
    val_auc = roc_auc_score(y_val, y_val_prob)
    test_auc = roc_auc_score(y_test, y_test_prob)

    val_precision = precision_score(y_val, y_val_pred)
    val_recall = recall_score(y_val, y_val_pred)
    val_f1 = f1_score(y_val, y_val_pred)

    test_precision = precision_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    wandb.log({
        'train_auc': train_auc,
        'val_auc': val_auc,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1,
        'test_auc': test_auc,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    })

    joblib.dump({'model': model, 'scaler': scaler}, os.path.join(save_dir, f'baseline_{model_type}_model.pkl'))

    print(f"\n{model_type.upper()}: Train {train_auc:.3f} | Val {val_auc:.3f} | Test {test_auc:.3f}")

    run.finish()

    return {
        'model': f'Baseline-{model_type}',
        'val_auc': val_auc,
        'test_auc': test_auc,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    }

BASELINE_SWEEP_CONFIG = {
    'method': 'bayes',
    'metric': {'name': 'val_auc', 'goal': 'maximize'},
    'parameters': {
        'model_type': {'values': ['xgboost', 'lightgbm']},
        'n_estimators': {'values': [100, 200, 300, 500]},
        'max_depth': {'values': [3, 5, 6, 8, 10]},
        'learning_rate': {'distribution': 'log_uniform_values', 'min': 0.01, 'max': 0.3}
    }
}

In [8]:
def run_baseline_single():
    print("\n" + "="*80)
    print("TRAINING BASELINE (SINGLE RUN)")
    print("="*80)

    all_ids = train_ids + val_ids + test_ids
    features = extract_baseline_features(observation_data, all_ids)

    X_train = features.loc[train_ids]
    X_val = features.loc[val_ids]
    X_test = features.loc[test_ids]

    y_train = pd.Series(train_labels, index=train_ids)
    y_val = pd.Series(val_labels, index=val_ids)
    y_test = pd.Series(test_labels, index=test_ids)

    result = train_baseline(X_train, X_val, X_test, y_train, y_val, y_test, SAVE_DIR)
    return result

def run_baseline_sweep(count=10):
    print("\n" + "="*80)
    print(f"BASELINE SWEEP ({count} runs)")
    print("="*80)

    all_ids = train_ids + val_ids + test_ids
    features = extract_baseline_features(observation_data, all_ids)

    X_train = features.loc[train_ids]
    X_val = features.loc[val_ids]
    X_test = features.loc[test_ids]

    y_train = pd.Series(train_labels, index=train_ids)
    y_val = pd.Series(val_labels, index=val_ids)
    y_test = pd.Series(test_labels, index=test_ids)

    def sweep_train():
        train_baseline(X_train, X_val, X_test, y_train, y_val, y_test, SAVE_DIR)

    sweep_id = wandb.sweep(BASELINE_SWEEP_CONFIG, project="churn-prediction")
    wandb.agent(sweep_id, function=sweep_train, count=count)

# LSTM

In [9]:
class SequenceDataset(Dataset):
    def __init__(self, sequences, labels, max_len=50):
        self.sequences = sequences
        self.labels = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        label = self.labels[idx]

        if len(seq) > self.max_len:
            seq = seq[-self.max_len:]
        else:
            pad = np.zeros((self.max_len - len(seq), seq.shape[1]))
            seq = np.vstack([pad, seq])

        mask = torch.ones(self.max_len)
        if len(self.sequences[idx]) < self.max_len:
            mask[:self.max_len - len(self.sequences[idx])] = 0

        return torch.FloatTensor(seq), torch.FloatTensor([label]), mask

class AttentionLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.3):
        super(AttentionLSTM, self).__init__()

        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers=num_layers,
            batch_first=True, dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )

        self.attention = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 1)
        )

    def forward(self, x, mask):
        lstm_out, _ = self.lstm(x)
        attn_weights = self.attention(lstm_out)
        attn_weights = attn_weights.masked_fill(mask.unsqueeze(-1) == 0, -1e9)
        attn_weights = F.softmax(attn_weights, dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        output = self.classifier(context)
        return torch.sigmoid(output), attn_weights

def create_lstm_sequences(df, user_ids):
    df = df.sort_values(['device_id', 'event_dt'])

    le_screen = LabelEncoder()
    le_feature = LabelEncoder()
    le_action = LabelEncoder()
    le_gender = LabelEncoder()

    df['screen_enc'] = le_screen.fit_transform(df['screen'].fillna('unknown'))
    df['feature_enc'] = le_feature.fit_transform(df['feature'].fillna('unknown'))
    df['action_enc'] = le_action.fit_transform(df['action'].fillna('unknown'))
    df['gender_enc'] = le_gender.fit_transform(df['gender'].fillna('unknown'))

    df['hour'] = df['event_dt'].dt.hour / 24.0
    df['day_of_week'] = df['event_dt'].dt.dayofweek / 7.0
    df['is_weekend'] = (df['event_dt'].dt.dayofweek >= 5).astype(float)
    df['time_diff'] = df.groupby('device_id')['event_dt'].diff().dt.total_seconds() / 3600
    df['time_diff'] = df['time_diff'].fillna(0).clip(0, 24) / 24.0

    df['age_norm'] = (df['age'] - df['age'].mean()) / df['age'].std()
    df['age_norm'] = df['age_norm'].fillna(0)

    feature_cols = [
        'screen_enc', 'feature_enc', 'action_enc', 'gender_enc',
        'hour', 'day_of_week', 'is_weekend', 'time_diff', 'age_norm'
    ]

    sequences = []
    sequence_user_ids = []

    for device_id in user_ids:
        user_data = df[df['device_id'] == device_id]
        if len(user_data) > 0:
            seq = user_data[feature_cols].values.astype(np.float32)
            sequences.append(seq)
            sequence_user_ids.append(device_id)

    return sequences, sequence_user_ids, len(feature_cols)

def train_lstm(train_loader, val_loader, test_loader, input_dim, save_dir, device='cuda', config=None):
    run = wandb.init(project="churn-prediction", config=config)
    config = wandb.config

    hidden_dim = getattr(config, 'hidden_dim', 128)
    num_layers = getattr(config, 'num_layers', 2)
    dropout = getattr(config, 'dropout', 0.3)
    learning_rate = getattr(config, 'learning_rate', 0.001)
    max_epochs = getattr(config, 'epochs', 30)

    model = AttentionLSTM(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout)
    model = model.to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3)

    best_val_auc = 0
    patience_counter = 0

    for epoch in range(max_epochs):
        model.train()
        train_preds, train_labels = [], []

        for sequences, labels, masks in train_loader:
            sequences, labels, masks = sequences.to(device), labels.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs, _ = model(sequences, masks)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.detach().cpu().numpy())

        model.eval()
        val_preds, val_labels = [], []

        with torch.no_grad():
            for sequences, labels, masks in val_loader:
                sequences, labels, masks = sequences.to(device), labels.to(device), masks.to(device)
                outputs, _ = model(sequences, masks)
                val_preds.extend(outputs.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        train_auc = roc_auc_score(np.array(train_labels).flatten(), np.array(train_preds).flatten())
        val_auc = roc_auc_score(np.array(val_labels).flatten(), np.array(val_preds).flatten())

        scheduler.step(val_auc)

        wandb.log({'epoch': epoch + 1, 'train_auc': train_auc, 'val_auc': val_auc})

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1:02d} | Train {train_auc:.3f} | Val {val_auc:.3f}")

        if val_auc > best_val_auc:
            best_val_auc = val_auc
            patience_counter = 0
            torch.save({
                'model_state_dict': model.state_dict(),
                'config': {'input_dim': input_dim, 'hidden_dim': hidden_dim, 'num_layers': num_layers, 'dropout': dropout}
            }, os.path.join(save_dir, 'lstm_model.pth'))
        else:
            patience_counter += 1
            if patience_counter >= 7:
                print(f"Early stopping at epoch {epoch+1}")
                break

    model.load_state_dict(torch.load(os.path.join(save_dir, 'lstm_model.pth'))['model_state_dict'])
    model.eval()
    test_preds, test_labels = [], []

    with torch.no_grad():
        for sequences, labels, masks in test_loader:
            sequences, masks = sequences.to(device), masks.to(device)
            outputs, _ = model(sequences, masks)
            test_preds.extend(outputs.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    test_preds = np.array(test_preds).flatten()
    test_labels = np.array(test_labels).flatten()
    test_pred_binary = (test_preds > 0.5).astype(int)

    test_auc = roc_auc_score(test_labels, test_preds)
    test_precision = precision_score(test_labels, test_pred_binary)
    test_recall = recall_score(test_labels, test_pred_binary)
    test_f1 = f1_score(test_labels, test_pred_binary)

    wandb.log({
        'test_auc': test_auc,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    })

    print(f"\nLSTM: Best Val {best_val_auc:.3f} | Test {test_auc:.3f}")

    run.finish()

    return {
        'model': 'LSTM',
        'val_auc': best_val_auc,
        'test_auc': test_auc,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    }

LSTM_SWEEP_CONFIG = {
    'method': 'bayes',
    'metric': {'name': 'val_auc', 'goal': 'maximize'},
    'parameters': {
        'hidden_dim': {'values': [64, 128, 256]},
        'num_layers': {'values': [1, 2, 3]},
        'dropout': {'distribution': 'uniform', 'min': 0.1, 'max': 0.5},
        'learning_rate': {'distribution': 'log_uniform_values', 'min': 0.0001, 'max': 0.01},
        'epochs': {'value': 30}
    }
}

In [10]:
def run_lstm_single():
    print("\n" + "="*80)
    print("TRAINING LSTM (SINGLE RUN)")
    print("="*80)

    train_seqs, train_seq_ids, input_dim = create_lstm_sequences(observation_data, train_ids)
    val_seqs, val_seq_ids, _ = create_lstm_sequences(observation_data, val_ids)
    test_seqs, test_seq_ids, _ = create_lstm_sequences(observation_data, test_ids)

    train_seq_labels = [churn_labels[uid] for uid in train_seq_ids]
    val_seq_labels = [churn_labels[uid] for uid in val_seq_ids]
    test_seq_labels = [churn_labels[uid] for uid in test_seq_ids]

    train_dataset = SequenceDataset(train_seqs, train_seq_labels, max_len=50)
    val_dataset = SequenceDataset(val_seqs, val_seq_labels, max_len=50)
    test_dataset = SequenceDataset(test_seqs, test_seq_labels, max_len=50)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    result = train_lstm(train_loader, val_loader, test_loader, input_dim, SAVE_DIR, device)
    return result

def run_lstm_sweep(count=20):
    print("\n" + "="*80)
    print(f"LSTM SWEEP ({count} runs)")
    print("="*80)

    train_seqs, train_seq_ids, input_dim = create_lstm_sequences(observation_data, train_ids)
    val_seqs, val_seq_ids, _ = create_lstm_sequences(observation_data, val_ids)
    test_seqs, test_seq_ids, _ = create_lstm_sequences(observation_data, test_ids)

    train_seq_labels = [churn_labels[uid] for uid in train_seq_ids]
    val_seq_labels = [churn_labels[uid] for uid in val_seq_ids]
    test_seq_labels = [churn_labels[uid] for uid in test_seq_ids]

    train_dataset = SequenceDataset(train_seqs, train_seq_labels, max_len=50)
    val_dataset = SequenceDataset(val_seqs, val_seq_labels, max_len=50)
    test_dataset = SequenceDataset(test_seqs, test_seq_labels, max_len=50)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def sweep_train():
        train_lstm(train_loader, val_loader, test_loader, input_dim, SAVE_DIR, device)

    sweep_id = wandb.sweep(LSTM_SWEEP_CONFIG, project="churn-prediction")
    wandb.agent(sweep_id, function=sweep_train, count=count)

# GNN

In [11]:
class GraphSAGEChurn(nn.Module):
    def __init__(self, in_channels, hidden_channels=128, num_layers=2, dropout=0.3):
        super(GraphSAGEChurn, self).__init__()

        self.convs = nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))

        for _ in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))

        self.dropout = dropout

        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 1)
        )

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < len(self.convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)

        out = self.classifier(x)
        return torch.sigmoid(out)

def build_random_graph(n_users, edges_per_node=5):
    edge_list = []
    for i in range(n_users):
        neighbors = np.random.choice(n_users, size=min(edges_per_node, n_users-1), replace=False)
        neighbors = neighbors[neighbors != i]
        for j in neighbors:
            edge_list.append([i, j])

    return torch.tensor(edge_list, dtype=torch.long).t().contiguous()

def train_gnn(data, train_mask, val_mask, test_mask, save_dir, device='cuda', config=None):
    run = wandb.init(project="churn-prediction", config=config)
    config = wandb.config

    hidden_channels = getattr(config, 'hidden_channels', 128)
    num_layers = getattr(config, 'num_layers', 2)
    dropout = getattr(config, 'dropout', 0.3)
    learning_rate = getattr(config, 'learning_rate', 0.01)
    weight_decay = getattr(config, 'weight_decay', 5e-4)
    max_epochs = getattr(config, 'epochs', 100)

    model = GraphSAGEChurn(in_channels=data.x.size(1), hidden_channels=hidden_channels, num_layers=num_layers, dropout=dropout)
    model = model.to(device)
    data = data.to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    best_val_auc = 0
    patience_counter = 0

    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index).squeeze()
        loss = criterion(out[train_mask], data.y[train_mask])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            out = model(data.x, data.edge_index).squeeze()

            train_pred = out[train_mask].cpu().numpy()
            train_true = data.y[train_mask].cpu().numpy()
            val_pred = out[val_mask].cpu().numpy()
            val_true = data.y[val_mask].cpu().numpy()

            train_auc = roc_auc_score(train_true, train_pred)
            val_auc = roc_auc_score(val_true, val_pred)

        wandb.log({'epoch': epoch + 1, 'train_auc': train_auc, 'val_auc': val_auc})

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1:03d} | Train {train_auc:.3f} | Val {val_auc:.3f}")

        if val_auc > best_val_auc:
            best_val_auc = val_auc
            patience_counter = 0
            torch.save({
                'model_state_dict': model.state_dict(),
                'config': {'in_channels': data.x.size(1), 'hidden_channels': hidden_channels, 'num_layers': num_layers, 'dropout': dropout}
            }, os.path.join(save_dir, 'gnn_model.pth'))
        else:
            patience_counter += 1
            if patience_counter >= 15:
                print(f"Early stopping at epoch {epoch+1}")
                break

    model.load_state_dict(torch.load(os.path.join(save_dir, 'gnn_model.pth'))['model_state_dict'])
    model.eval()

    with torch.no_grad():
        out = model(data.x, data.edge_index).squeeze()
        test_pred = out[test_mask].cpu().numpy()
        test_true = data.y[test_mask].cpu().numpy()

    test_pred_binary = (test_pred > 0.5).astype(int)
    test_auc = roc_auc_score(test_true, test_pred)
    test_precision = precision_score(test_true, test_pred_binary)
    test_recall = recall_score(test_true, test_pred_binary)
    test_f1 = f1_score(test_true, test_pred_binary)

    wandb.log({
        'test_auc': test_auc,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    })

    print(f"\nGNN: Best Val {best_val_auc:.3f} | Test {test_auc:.3f}")

    run.finish()

    return {
        'model': 'GNN',
        'val_auc': best_val_auc,
        'test_auc': test_auc,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    }

GNN_SWEEP_CONFIG = {
    'method': 'bayes',
    'metric': {'name': 'val_auc', 'goal': 'maximize'},
    'parameters': {
        'hidden_channels': {'values': [64, 128, 256]},
        'num_layers': {'values': [1, 2, 3]},
        'dropout': {'distribution': 'uniform', 'min': 0.1, 'max': 0.5},
        'learning_rate': {'distribution': 'log_uniform_values', 'min': 0.001, 'max': 0.1},
        'weight_decay': {'distribution': 'log_uniform_values', 'min': 1e-5, 'max': 1e-3},
        'epochs': {'value': 100}
    }
}

In [12]:
def run_gnn_single():
    print("\n" + "="*80)
    print("TRAINING GNN (SINGLE RUN)")
    print("="*80)

    all_ids = train_ids + val_ids + test_ids
    features = extract_baseline_features(observation_data, all_ids)
    X_scaled = StandardScaler().fit_transform(features.loc[all_ids])

    edge_index = build_random_graph(len(all_ids), edges_per_node=5)

    user_id_to_idx = {uid: idx for idx, uid in enumerate(all_ids)}
    train_mask = torch.zeros(len(all_ids), dtype=torch.bool)
    val_mask = torch.zeros(len(all_ids), dtype=torch.bool)
    test_mask = torch.zeros(len(all_ids), dtype=torch.bool)

    for uid in train_ids:
        train_mask[user_id_to_idx[uid]] = True
    for uid in val_ids:
        val_mask[user_id_to_idx[uid]] = True
    for uid in test_ids:
        test_mask[user_id_to_idx[uid]] = True

    x = torch.FloatTensor(X_scaled)
    y = torch.FloatTensor([churn_labels[uid] for uid in all_ids])
    data = Data(x=x, edge_index=edge_index, y=y)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    result = train_gnn(data, train_mask, val_mask, test_mask, SAVE_DIR, device)

    return result

def run_gnn_sweep(count=15):
    print("\n" + "="*80)
    print(f"GNN SWEEP ({count} runs)")
    print("="*80)

    all_ids = train_ids + val_ids + test_ids
    features = extract_baseline_features(observation_data, all_ids)
    X_scaled = StandardScaler().fit_transform(features.loc[all_ids])

    edge_index = build_random_graph(len(all_ids), edges_per_node=5)

    user_id_to_idx = {uid: idx for idx, uid in enumerate(all_ids)}
    train_mask = torch.zeros(len(all_ids), dtype=torch.bool)
    val_mask = torch.zeros(len(all_ids), dtype=torch.bool)
    test_mask = torch.zeros(len(all_ids), dtype=torch.bool)

    for uid in train_ids:
        train_mask[user_id_to_idx[uid]] = True
    for uid in val_ids:
        val_mask[user_id_to_idx[uid]] = True
    for uid in test_ids:
        test_mask[user_id_to_idx[uid]] = True

    x = torch.FloatTensor(X_scaled)
    y = torch.FloatTensor([churn_labels[uid] for uid in all_ids])
    data = Data(x=x, edge_index=edge_index, y=y)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def sweep_train():
        train_gnn(data, train_mask, val_mask, test_mask, SAVE_DIR, device)

    sweep_id = wandb.sweep(GNN_SWEEP_CONFIG, project="churn-prediction")
    wandb.agent(sweep_id, function=sweep_train, count=count)

# Split and save

In [13]:
df = pd.read_csv(DATA_PATH)
print(f"Loaded {len(df):,} events, {df['device_id'].nunique():,} users")

Loaded 3,360,650 events, 179,976 users


In [14]:
observation_data, churn_labels = create_temporal_dataset(df, OBSERVATION_DAYS, PREDICTION_DAYS)
split_data = create_unified_split(observation_data, churn_labels, SAVE_DIR)

train_ids = split_data['train_ids']
val_ids = split_data['val_ids']
test_ids = split_data['test_ids']
train_labels = split_data['train_labels']
val_labels = split_data['val_labels']
test_labels = split_data['test_labels']

print(f"Train: {len(train_ids)} users, Val: {len(val_ids)} users, Test: {len(test_ids)} users")


Observation: 2025-09-01 to 2025-09-30
Prediction: 2025-09-30 to 2025-10-31
Users: 135,860 | Churned: 59,203 (43.6%)

Split: Train 81,516 | Val 27,172 | Test 27,172
Train: 81516 users, Val: 27172 users, Test: 27172 users


# Test 3 models and compare

In [17]:
!nvidia-smi

Wed Dec 17 03:54:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:0B:00.0  On |                  N/A |
|  0%   40C    P8             17W /  350W |     433MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
def run_all_single():
    print("\n" + "="*80)
    print("TRAINING ALL MODELS (SINGLE RUNS)")
    print("="*80)

    results = []

    results.append(run_baseline_single())
    results.append(run_lstm_single())
    results.append(run_gnn_single())

    results_df = pd.DataFrame(results)

    print("\n" + "="*80)
    print("FINAL RESULTS")
    print("="*80 + "\n")
    print(results_df.to_string(index=False))

    results_df.to_csv(os.path.join(SAVE_DIR, 'final_results.csv'), index=False)

    best_model = results_df.loc[results_df['test_auc'].idxmax()]
    print(f"\n🏆 Best: {best_model['model']} (Test AUC: {best_model['test_auc']:.4f})")

    return results_df

In [19]:
run_all_single()


TRAINING ALL MODELS (SINGLE RUNS)

TRAINING BASELINE (SINGLE RUN)



XGBOOST: Train 0.771 | Val 0.724 | Test 0.725


test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▁
val_auc,▁
val_f1,▁
val_precision,▁
val_recall,▁
test_auc,0.72533
test_f1,0.65398



TRAINING LSTM (SINGLE RUN)
Using device: cuda


Epoch 05 | Train 0.697 | Val 0.710
Epoch 10 | Train 0.705 | Val 0.717
Epoch 15 | Train 0.709 | Val 0.720
Epoch 20 | Train 0.710 | Val 0.709
Epoch 25 | Train 0.710 | Val 0.709
Epoch 30 | Train 0.720 | Val 0.724

LSTM: Best Val 0.724 | Test 0.722


...


TRAINING GNN (SINGLE RUN)
Using device: cuda


Epoch 010 | Train 0.706 | Val 0.708
Epoch 020 | Train 0.710 | Val 0.711
Epoch 030 | Train 0.712 | Val 0.713
Epoch 040 | Train 0.714 | Val 0.714
Epoch 050 | Train 0.716 | Val 0.715
Epoch 060 | Train 0.719 | Val 0.717
Epoch 070 | Train 0.717 | Val 0.715
Early stopping at epoch 79

GNN: Best Val 0.717 | Test 0.718


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▁▁▁▃▄▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█████▇██▇█▇████
val_auc,▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████████
epoch,79
test_auc,0.71822
test_f1,0.60825
test_precision,0.61186



FINAL RESULTS

           model  val_auc  test_auc  test_precision  test_recall  test_f1
Baseline-xgboost 0.724066  0.725331        0.588414     0.736002 0.653985
            LSTM 0.723688  0.722296        0.611871     0.631197 0.621383
             GNN 0.717367  0.718221        0.611861     0.604679 0.608249

🏆 Best: Baseline-xgboost (Test AUC: 0.7253)


,model,val_auc,test_auc,test_precision,test_recall,test_f1
0,Baseline-xgboost,0.724066,0.725331,0.588414,0.736002,0.653985
1,LSTM,0.723688,0.722296,0.611871,0.631197,0.621383
2,GNN,0.717367,0.718221,0.611861,0.604679,0.608249


# SWEEPS (HYPERPARAMETER TUNING)

In [20]:
run_baseline_sweep(count=10)


BASELINE SWEEP (10 runs)
Create sweep with ID: pk7vkeqn
Sweep URL: https://wandb.ai/katrinpochtar/churn-prediction/sweeps/pk7vkeqn


wandb: Agent Starting Run: 910oo061 with config:
wandb: 	learning_rate: 0.02794859513956267
wandb: 	max_depth: 8
wandb: 	model_type: lightgbm
wandb: 	n_estimators: 200



LIGHTGBM: Train 0.739 | Val 0.727 | Test 0.727


test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▁
val_auc,▁
val_f1,▁
val_precision,▁
val_recall,▁
test_auc,0.72654
test_f1,0.65605


wandb: Agent Starting Run: cxgs33l0 with config:
wandb: 	learning_rate: 0.0927025430598389
wandb: 	max_depth: 10
wandb: 	model_type: lightgbm
wandb: 	n_estimators: 500


...


XGBOOST: Train 0.753 | Val 0.724 | Test 0.724


test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▁
val_auc,▁
val_f1,▁
val_precision,▁
val_recall,▁
test_auc,0.72408
test_f1,0.65685


wandb: Agent Starting Run: gf2zbsjl with config:
wandb: 	learning_rate: 0.17294065810406992
wandb: 	max_depth: 3
wandb: 	model_type: lightgbm
wandb: 	n_estimators: 100



LIGHTGBM: Train 0.732 | Val 0.726 | Test 0.726


test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▁
val_auc,▁
val_f1,▁
val_precision,▁
val_recall,▁
test_auc,0.72558
test_f1,0.65653


In [21]:
run_lstm_sweep(count=20)


LSTM SWEEP (20 runs)
Create sweep with ID: iimvxwhr
Sweep URL: https://wandb.ai/katrinpochtar/churn-prediction/sweeps/iimvxwhr


wandb: Agent Starting Run: 98jdu8te with config:
wandb: 	dropout: 0.12084462228588304
wandb: 	epochs: 30
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.00625821406522273
wandb: 	num_layers: 1


Epoch 05 | Train 0.669 | Val 0.684
Early stopping at epoch 9

LSTM: Best Val 0.695 | Test 0.695


epoch,▁▂▃▄▅▅▆▇█
test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▁▅▅▄▄▄▆▆█
val_auc,▅█▄▃▄▁▄▇▄
epoch,9
test_auc,0.69506
test_f1,0.58833
test_precision,0.59589


wandb: Agent Starting Run: 8vagytl4 with config:
wandb: 	dropout: 0.45649425272167377
wandb: 	epochs: 30
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.0002866502291968808
wandb: 	num_layers: 1


...

Epoch 05 | Train 0.703 | Val 0.708
Epoch 10 | Train 0.709 | Val 0.717
Epoch 15 | Train 0.714 | Val 0.719
Epoch 20 | Train 0.715 | Val 0.723
Epoch 25 | Train 0.721 | Val 0.725
Epoch 30 | Train 0.723 | Val 0.725

LSTM: Best Val 0.725 | Test 0.723


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▁▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████
val_auc,▁▃▄▅▅▆▆▆▆▇▇▆▇▇▇▇▇▇▇███▇███████
epoch,30
test_auc,0.72273
test_f1,0.62731
test_precision,0.60879


wandb: Agent Starting Run: 6voipaw6 with config:
wandb: 	dropout: 0.30331091665339277
wandb: 	epochs: 30
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.00011005547571994892
wandb: 	num_layers: 1


Epoch 05 | Train 0.702 | Val 0.711
Epoch 10 | Train 0.710 | Val 0.715
Epoch 15 | Train 0.713 | Val 0.719
Epoch 20 | Train 0.716 | Val 0.721
Epoch 25 | Train 0.721 | Val 0.723
Epoch 30 | Train 0.722 | Val 0.723

LSTM: Best Val 0.723 | Test 0.721


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▁▄▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████████
val_auc,▁▄▅▆▆▆▆▆▆▇▇▇▇▇▇▇█▇████████████
epoch,30
test_auc,0.72099
test_f1,0.61521
test_precision,0.61384


In [22]:
run_gnn_sweep(count=15)


GNN SWEEP (15 runs)
Create sweep with ID: fcwt7ups
Sweep URL: https://wandb.ai/katrinpochtar/churn-prediction/sweeps/fcwt7ups


wandb: Agent Starting Run: 5pnb7pmq with config:
wandb: 	dropout: 0.4843811920524954
wandb: 	epochs: 100
wandb: 	hidden_channels: 256
wandb: 	learning_rate: 0.08226483188419124
wandb: 	num_layers: 2
wandb: 	weight_decay: 1.771897664335087e-05


Epoch 010 | Train 0.500 | Val 0.500
Early stopping at epoch 16

GNN: Best Val 0.570 | Test 0.565


epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,█▁▁▆▄▅▅▅▅▅▅▅▅▅▅▅
val_auc,█▁▁▆▄▅▅▅▅▅▅▅▅▅▅▅
epoch,16
test_auc,0.5651
test_f1,0
test_precision,0


wandb: Agent Starting Run: x6i1dvr0 with config:
wandb: 	dropout: 0.11799151259717124
wandb: 	epochs: 100
wandb: 	hidden_channels: 256
wandb: 	learning_rate: 0.007311143920361205
wandb: 	num_layers: 1
wandb: 	weight_decay: 0.0009427576617795124


...

Epoch 010 | Train 0.684 | Val 0.689
Epoch 020 | Train 0.704 | Val 0.703
Epoch 030 | Train 0.711 | Val 0.712
Epoch 040 | Train 0.713 | Val 0.713
Epoch 050 | Train 0.715 | Val 0.714
Epoch 060 | Train 0.717 | Val 0.715
Epoch 070 | Train 0.719 | Val 0.717
Epoch 080 | Train 0.722 | Val 0.717
Epoch 090 | Train 0.723 | Val 0.719
Epoch 100 | Train 0.725 | Val 0.719

GNN: Best Val 0.719 | Test 0.719


epoch,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▃▁▇▇▇███████████████████████████████████
val_auc,▄▁▁▇▇███████████████████████████████████
epoch,100
test_auc,0.71877
test_f1,0.6249
test_precision,0.60471


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: jhk3pggu with config:
wandb: 	dropout: 0.1223538818335972
wandb: 	epochs: 100
wandb: 	hidden_channels: 64
wandb: 	learning_rate: 0.01872072280531123
wandb: 	num_layers: 3
wandb: 	weight_decay: 0.0006491273015811683


Epoch 010 | Train 0.702 | Val 0.705
Epoch 020 | Train 0.710 | Val 0.712
Epoch 030 | Train 0.713 | Val 0.713
Epoch 040 | Train 0.715 | Val 0.715
Epoch 050 | Train 0.717 | Val 0.716
Epoch 060 | Train 0.719 | Val 0.717
Epoch 070 | Train 0.721 | Val 0.718
Epoch 080 | Train 0.722 | Val 0.719
Epoch 090 | Train 0.725 | Val 0.720
Epoch 100 | Train 0.716 | Val 0.716

GNN: Best Val 0.720 | Test 0.719


epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_auc,▁▅▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████▇
val_auc,▁▁▂▃▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████▇▇▇
epoch,100
test_auc,0.71895
test_f1,0.6451
test_precision,0.59114
